In [ ]:
import msgpackrpc

Import the `msgpackrpc-python` library.

We need the version from [with_tornado_453](https://github.com/xaedes/msgpack-rpc-python/tree/with_tornado_453) branch.

In [ ]:
m4_proxy_register = msgpackrpc.Address('m4-proxy', 5000)
client = msgpackrpc.Client(m4_proxy_register)

Create the RPC client.

> **TL;DR** The `m4-proxy` hostname points to `host-gateway`, that Docker will resolve to the current address of the X8 as network node.

> **NOTE**: The `m4-proxy` is a service that runs as a native service inside the X8's Linux microPlaftorm. Furthermore `m4-proxy` forwards the RPC calls from the M4 only to RPC server running on localhost – ie on the very X8 – and discards the host part of the `host:port` TCP address pair.
Because this Jupyter instance – as wall as all the customer applications – runs inside a Docker container, we need a way to reach the X8 as a network host from the Docker container. This is done by registering a special hostname via the `--add-host` parameter from the `docker` CLI or with the `extra-hosts` key in the `docker-compose.yml` file.



In [ ]:
client.call('register', 5015, ['tty'])

Register this Python script as an RPC server that expose the `tty` service on the `5015` port.

In [ ]:
class SerialRPC(object):
    def tty(self, msg):
        print(f'{msg.decode()}', end=None)

Create a class that holds the implementations for all the procedure that this service exposes.

> **NOTE**: The `tty` procedure is a special procedure that gets called by the Arduino SerialRPC library.
When an Arduino Sketch includes the `<SerialRPC.h>` header, all the `Serial` calls are actually RPC calls to the `tty` procedure, passing the string to print as parameter.

In [ ]:
try:
    rpc_server = msgpackrpc.Server(SerialRPC())
    rpc_server.listen(msgpackrpc.Address("localhost", 5015))
    rpc_server.start()
except KeyboardInterrupt:
    rpc_server.stop()
    rpc_server.close()

Create the RPC Server and wait incoming RPC calls